In [1]:
import os
import json
import pandas as pd
import requests
from pandas.core.frame import DataFrame

import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
df_pr1 = pd.read_csv('Data/all_runs_in_project_PRJNA485797.tsv', sep="	" , index_col=False)
df_pr1.describe()

,nr. reads seqeunced,host age in years,BMI,recent antibiotics use,QC status
count,13.000000,56.000000,13.000000,0.0,219.000000
mean,34847.384615,64.053571,21.299231,NaN,0.968037
std,58549.152454,19.156219,9.882347,NaN,0.176306
min,1.000000,0.000000,0.000000,NaN,0.000000
25%,12989.000000,59.000000,21.300000,NaN,1.000000
50%,23205.000000,67.000000,24.410000,NaN,1.000000
75%,25836.000000,78.000000,26.090000,NaN,1.000000
max,227161.000000,85.000000,30.850000,NaN,1.000000


In [3]:
df_QC1 = df_pr1.loc[df_pr1['QC status'] == 1.0]
#df_phen_health = df_QC1.loc[df_QC1['associated phenotype'] == 'D006262'] #healthy
df_QC1.head()

,project ID,run ID,sample ID,experiment type,nr. reads seqeunced,instrument model,country,collection date,gender,host age in years,diet,BMI,recent antibiotics use,QC status
0,PRJEB11419,ERR1090510,ERS940223,Amplicon,26067.0,Illumina MiSeq,United States of America,2015-03-18,Male,31.0,Omnivore,30.11,NaN,1
2,PRJEB11419,ERR1160800,ERS994317,Amplicon,227161.0,Illumina HiSeq 2000,United States of America,2015-10-23,Female,53.0,Omnivore,26.09,NaN,1
7,PRJEB11419,ERR1843463,ERS1561262,Amplicon,23205.0,Illumina MiSeq,United States of America,2015-05-18,Male,0.0,Omnivore,24.41,NaN,1
8,PRJEB11419,ERR1844528,ERS1561262,Amplicon,23303.0,Illumina MiSeq,United States of America,2015-05-18,Male,0.0,Omnivore,24.41,NaN,1
10,PRJEB11419,ERR2091834,ERS1865177,Amplicon,25836.0,Illumina MiSeq,United Kingdom,2016-12-07,Male,56.0,Omnivore,23.72,NaN,1


In [4]:
df_runs = df_QC1['run ID']
df_runs.describe()

count            212
unique           212
top       ERR1090510
freq               1
Name: run ID, dtype: object

In [5]:
df_only_runs = pd.DataFrame({'Runs': df_runs.values})

In [6]:
df_only_runs.describe()

,Runs
count,212
unique,212
top,ERR1090510
freq,1


In [ ]:
all_flattened_runs = []
identifier = 'Healthy'

In [8]:


for index, run_id in df_only_runs["Runs"].items():
    try:
        query = {"run_id":run_id}  
        url = 'https://gmrepo.humangut.info/api/getFullTaxonomicProfileByRunID'
        data = requests.post(url, data=json.dumps(query)).json()

        ## --get run List
        run = data.get("run")

        ## --get DataFrames
        species = DataFrame(data.get("species"))
        species = species.drop(columns = ['taxon_rank_level', 'scientific_name']) #drop useless info

        #species = species[species['ncbi_taxon_id'] != -1]   #drop missing taxons
        flat = species.set_index('ncbi_taxon_id')['relative_abundance'].T
        flat = flat.rename_axis(None).to_frame().T
        flat.insert(0, 'Run', run_id)
        all_flattened_runs.append(flat)
    except Exception as e:
        print(f"Error processing run {run}: {e}")



Error processing run {'project_id': 'PRJNA633959', 'original_sample_description': None, 'run_id': 'SRR11825857', 'experiment_type': 'Amplicon', 'instrument_model': 'Illumina MiSeq', 'nr_reads_sequenced': None, 'host_age': None, 'sex': None, 'BMI': None, 'country': 'China', 'longitude': None, 'latitude': None, 'loaded_uid': 66356, 'QCStatus': 1, 'QCMessage': 'p-trim-left-f 0;p-trim-left-r 19;p-trunc-len-f 290;p-trunc-len-r 260', 'Original_Project_description': 'With the development of the next-generation sequencing techniques, the higher sequencing depth and coverage made it possible to decipher those unknown key functional taxa in Chinese AD patients. In our present study, the AD associated fecal microbiota were analyzed from AD patients and matched healthy controls from Lishui by 16S rRNA gene high-throughput sequencing platform, which will provide novel targets for the non-invasive diagnosis and personalized treatment of AD as well as the development of tailor-made probiotics designe

KeyboardInterrupt: 

In [ ]:
final_df = pd.concat(all_flattened_runs, ignore_index=True)
#final_df = pd.concat([df_only_runs.reset_index(drop=True), final_df], axis=1)
final_df = final_df.fillna(0)
final_df[identifier] = 1

C:\Users\Daria\AppData\Local\Temp\ipykernel_10944\3439001221.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[identifier] = 1


In [ ]:
final_df.head()

,Run,821,46506,562,1755626,28116,46503,39485,817,349096,...,995,999468,36844,69823,76832,1305,58180,467997,652716,Healthy
0,SRR7690035,18.960600,14.236300,13.555000,10.40170,6.634150,3.000630,2.807040,2.356580,1.612,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,SRR7690037,7.032240,0.391292,0.000000,6.74015,0.259025,0.347203,0.347203,0.424359,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,SRR7690038,18.984600,0.013051,0.047853,8.92678,16.070000,0.796102,0.004350,0.021751,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,SRR7690042,15.365700,1.167480,0.005235,7.82158,1.036590,1.649130,1.026120,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,SRR7690045,0.130682,6.477270,0.000000,12.95450,4.136360,0.011364,1.335230,1.392050,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
final_df.to_csv('Data/Raw_runs/healthy_runs_PRJNA485797', index = False)